# Predicting Hospital Admissions using Claims Data

#### *Please note that this project is still in progress*

## Introduction

Health informatics can carry significant impact with regards to costs and availability of services. The Heritage Health Competition was a past data competition hosted on Kaggle. Participants use available patient data to predict which patients are more likely to experience readmission. 

In this project, I use the past datasets to conduct data cleaning, exploratory data analysis, modeling, and appropriate predictive analysis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Wrangling 

The datasets were released via Kaggle in CSV formats. They contain many instances of incomplete cases and require extensive cleaning. The tables were pulled from a relational database, in which the member id is the primary field linking tables. Therefore, joins are required; the **members** and **target** tables have one-to-one relationships, they can be merged using left and/or inner joins. The **drugs** and **labs** tables have a one-to-many relationship with the member table, as they contain records on a yearly basis.

In [2]:
#Data Import

claims = pd.read_csv('HHP_release3/Claims.csv')
drugs = pd.read_csv('HHP_release3/DrugCount.csv')
labs = pd.read_csv('HHP_release3/LabCount.csv')
members = pd.read_csv('HHP_release3/Members.csv')
target = pd.read_csv('HHP_release3/Target.csv')
year2 = pd.read_csv('HHP_release3/DaysInHospital_Y2.csv')
year3 = pd.read_csv('HHP_release3/DaysInHospital_Y3.csv')

In [37]:
members.head()

,MemberID,AgeAtFirstClaim,Sex
0,14723353,70-79,M
1,75706636,70-79,M
2,17320609,70-79,M
3,69690888,40-49,M
4,33004608,0-9,M


In [40]:
member_dat = pd.merge(members, target, how='left', on='MemberID')
member_dat.head()

,MemberID,AgeAtFirstClaim,Sex,ClaimsTruncated,DaysInHospital
0,14723353,70-79,M,0.0,NaN
1,75706636,70-79,M,NaN,NaN
2,17320609,70-79,M,0.0,NaN
3,69690888,40-49,M,NaN,NaN
4,33004608,0-9,M,NaN,NaN


In [50]:
member_info = pd.merge(member_dat, drugs, how='left', on= 'MemberID')
member_info = pd.merge(member_info, labs, how='left', on=  ['MemberID','Year', 'DSFS'])
member_info = member_info.sort_values(by=['MemberID', 'Year', 'DSFS'], ignore_index = True)
member_info.head()

,MemberID,AgeAtFirstClaim,Sex,ClaimsTruncated,DaysInHospital,Year,DSFS,DrugCount,LabCount
0,4,0-9,M,NaN,NaN,NaN,NaN,NaN,NaN
1,210,30-39,NaN,0.0,NaN,Y1,0- 1 month,2,NaN
2,210,30-39,NaN,0.0,NaN,Y1,3- 4 months,2,NaN
3,210,30-39,NaN,0.0,NaN,Y1,4- 5 months,1,NaN
4,210,30-39,NaN,0.0,NaN,Y3,5- 6 months,2,NaN


In [51]:
claims = claims.sort_values(by=['MemberID', 'Year', 'DSFS'], ignore_index = True)
claims.head()

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,4,994608.0,851052.0,31106.0,Y2,Pediatrics,Office,43,NaN,0- 1 month,RESPR4,0,EM,0
1,210,9121540.0,523791.0,37508.0,Y1,Emergency,Urgent Care,57,2 days,0- 1 month,GIOBSENT,0,SDS,0
2,210,3554799.0,423477.0,18880.0,Y1,Emergency,Urgent Care,162+,NaN,0- 1 month,GIOBSENT,0,EM,0
3,210,8448244.0,122401.0,37508.0,Y1,Internal,Office,151,NaN,0- 1 month,GYNEC1,0,MED,0
4,210,7053364.0,240043.0,37508.0,Y1,Laboratory,Independent Lab,22,NaN,1- 2 months,MSC2a3,0,PL,0


In [76]:
members_conditions = claims.drop_duplicates(subset = ['MemberID', 'Year', 'PrimaryConditionGroup'])
members_conditions = members_conditions[['MemberID', 'Year', 'DSFS', 'PrimaryConditionGroup']]
members_conditions.head()

,MemberID,Year,DSFS,PrimaryConditionGroup
0,4,Y2,0- 1 month,RESPR4
1,210,Y1,0- 1 month,GIOBSENT
3,210,Y1,0- 1 month,GYNEC1
4,210,Y1,1- 2 months,MSC2a3
6,210,Y1,3- 4 months,PRGNCY


In [77]:
conditions=pd.pivot_table(members_conditions, index=['MemberID','Year','DSFS'], columns='PrimaryConditionGroup', aggfunc=len)
conditions = conditions.fillna(0)
conditions = conditions.reset_index()
conditions.head()

PrimaryConditionGroup,MemberID,Year,DSFS,AMI,APPCHOL,ARTHSPIN,CANCRA,CANCRB,CANCRM,CATAST,...,RENAL2,RENAL3,RESPR4,ROAMI,SEIZURE,SEPSIS,SKNAUT,STROKE,TRAUMA,UTI
0,4,Y2,0- 1 month,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,210,Y1,0- 1 month,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,210,Y1,1- 2 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,210,Y1,3- 4 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,210,Y2,0- 1 month,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
dat = pd.merge(member_info, conditions, how='inner', on= ['MemberID','Year', 'DSFS'])
dat.head()

,MemberID,AgeAtFirstClaim,Sex,ClaimsTruncated,DaysInHospital,Year,DSFS,DrugCount,LabCount,AMI,...,RENAL2,RENAL3,RESPR4,ROAMI,SEIZURE,SEPSIS,SKNAUT,STROKE,TRAUMA,UTI
0,210,30-39,NaN,0.0,NaN,Y1,0- 1 month,2,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,210,30-39,NaN,0.0,NaN,Y1,3- 4 months,2,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3197,0-9,F,0.0,NaN,Y1,1- 2 months,1,NaN,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3197,0-9,F,0.0,NaN,Y2,0- 1 month,2,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3197,0-9,F,0.0,NaN,Y2,1- 2 months,1,2,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3713,40-49,F,NaN,NaN,Y2,0- 1 month,6,NaN,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3713,40-49,F,NaN,NaN,Y2,1- 2 months,6,1,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,3741,70-79,F,NaN,NaN,Y2,0- 1 month,3,5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,3889,NaN,F,NaN,NaN,Y1,0- 1 month,3,10+,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
9,4048,50-59,M,0.0,NaN,Y3,1- 2 months,1,10+,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [87]:
dat['DaysInHospital'] = dat['DaysInHospital'].fillna(0)
dat['ClaimsTruncated'] = dat['ClaimsTruncated'].fillna(0)
dat['LabCount'] = dat['LabCount'].fillna(0)
dat['DrugCount'] = dat['DrugCount'].fillna(0)

In [89]:
dat.head()

,MemberID,AgeAtFirstClaim,Sex,ClaimsTruncated,DaysInHospital,Year,DSFS,DrugCount,LabCount,AMI,...,RENAL2,RENAL3,RESPR4,ROAMI,SEIZURE,SEPSIS,SKNAUT,STROKE,TRAUMA,UTI
0,210,30-39,NaN,0.0,0.0,Y1,0- 1 month,2,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,210,30-39,NaN,0.0,0.0,Y1,3- 4 months,2,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3197,0-9,F,0.0,0.0,Y1,1- 2 months,1,0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3197,0-9,F,0.0,0.0,Y2,0- 1 month,2,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3197,0-9,F,0.0,0.0,Y2,1- 2 months,1,2,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Feature Selection

One aspect of this project, which may differ from how other participants approached the challenge, entails my experience as a hospital volunteer, a public health student, and later a research assistant. Based on this, rather than employing forward or backward stepwise model building, I will be deliberately selecting features that have documented impacts on health. 

One feature that I will be constructing is an SES categorical variable, derived from the pay delay field. Pay delays can be the result of financial hardship, as I've learned through first hand experience. Socioeconomic status is a key determinant of health and will therefore be included in model building.

In [102]:
pay_delay = claims[['MemberID', 'PayDelay']]
pay_delay = pay_delay.dropna(axis = 1)

In [103]:
pay_delay.head()

,MemberID,PayDelay
0,4,43
1,210,57
2,210,162+
3,210,151
4,210,22


In [108]:
pay_delay.PayDelay = pd.to_numeric(pay_delay.PayDelay, downcast='float', errors='coerce')
pay_delay = pay_delay.fillna(163.0)

In [115]:
pay_delay2 = pay_delay[pay_delay.PayDelay > 90]
pay_delay2 = pay_delay2.drop_duplicates(subset = ['MemberID', 'PayDelay'])
pay_delay2.shape

(154212, 2)

In [120]:
pay_delay3 = pay_delay2.groupby(by="MemberID").count()
pay_delay3 = pay_delay3[pay_delay3.PayDelay > 2]
pay_delay3['low_SES'] = 1

### Merging Datasets Back Together

In [125]:
dat = pd.merge(dat, pay_delay3, how='left', on= ['MemberID'])
dat = dat.drop(columns = ['PayDelay'])
dat['low_SES'] = dat['low_SES'].fillna(0)
dat.LabCount = pd.to_numeric(dat.LabCount, downcast='float', errors='coerce')
dat.DrugCount = pd.to_numeric(dat.DrugCount, downcast='float', errors='coerce')

In [168]:
dat.head()

,MemberID,AgeAtFirstClaim,Sex,ClaimsTruncated,DaysInHospital,Year,DSFS,DrugCount,LabCount,AMI,...,RENAL3,RESPR4,ROAMI,SEIZURE,SEPSIS,SKNAUT,STROKE,TRAUMA,UTI,low_SES
0,210,30-39,NaN,0.0,0.0,Y1,0- 1 month,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,210,30-39,NaN,0.0,0.0,Y1,3- 4 months,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3197,0-9,F,0.0,0.0,Y1,1- 2 months,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3197,0-9,F,0.0,0.0,Y2,0- 1 month,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3197,0-9,F,0.0,0.0,Y2,1- 2 months,1.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
indices = list(range(7,55,1))
indices.append(5)

In [186]:
X = dat.iloc[:,indices]
Y = dat.DaysInHospital

In [187]:
X = pd.get_dummies(data=X, drop_first=True)
X = X.dropna(axis = 1)